In [1]:
# load module
import xarray as xr
import numpy as np

In [2]:
# compute closest ocean coastal cell
def get_dist_ocean_point_1d(xlon, ylat, lon1d, lat1d):
    '''Haversine formula'''
    earth_radius_miles = 6371.0
    dlat = lat1d[:] - ylat
    dlon = lon1d[:] - xlon
    a = np.square(np.sin(dlat/2.0)) + np.cos(ylat) * np.cos(lat1d[:]) * np.square(np.sin(dlon/2.0))
    great_circle_distance = 2 * np.arcsin(np.sqrt(a))
    d = earth_radius_miles * great_circle_distance
    return d

In [3]:
cfin ='BedMachineAntarctica-v3_ll.nc'
cfout='eANT025.L121_icbh_b0.6.1_v0.0.nc'

# ## Input data
print('open input file')
ds_in=xr.open_dataset(cfin)

open input file


In [4]:
print('get coastal trg mask')
mask_in=ds_in.msk_ice.values[:,:]
mask_coast_in=np.zeros(shape=mask_in.shape)
mask_coast_in[1:-1,1:-1]= ( mask_in[1:-1,1:-1] + 
                            mask_in[0:-2,1:-1] + mask_in[2::,1:-1] + mask_in[1:-1,0:-2] + mask_in[1:-1,2::] +
                            mask_in[0:-2,0:-2] + mask_in[2::,2::]  + mask_in[2::,0:-2]  + mask_in[0:-2,2::] ) * mask_in[1:-1,1:-1]
mask_coast_in[(mask_coast_in > 1) & (mask_coast_in < 9)] = 10
mask_coast_in[mask_coast_in!=10]=np.nan
mask_coast_in=mask_coast_in.astype(np.float32)
mask_coast_in[mask_coast_in==10]=1

get coastal trg mask


In [5]:
print('get BedMachine coastal input points')
ij_lst=np.where(mask_coast_in == 1)
print(len(ij_lst[0]))

get BedMachine coastal input points
472082


In [6]:
def get_1d_data(ds,ptslst):
    print('get lon 1d')
    lon1d=np.radians(ds.lon.values[ptslst[0],ptslst[1]])
    print('get lat 1d')
    lat1d=np.radians(ds.lat.values[ptslst[0],ptslst[1]])
    print('get thickness 1d')
    h1d=ds.thickness.values[ptslst[0],ptslst[1]]
    print('get mask ice')
    msk1d=ds.msk_ice.values[ptslst[0],ptslst[1]]
    return lon1d,lat1d,h1d,msk1d
lon_in_1d,lat_in_1d,h_in_1d,msk_in_1d=get_1d_data(ds_in,ij_lst)

get lon 1d
get lat 1d
get thickness 1d
get mask ice


In [8]:
# ## Output data
cftrg='eANT025.L121_mesh_mask_b0.6.1.nc'

print('open output file')
# define coastal mask
ds_trg=xr.open_dataset(cftrg)

lon_trg=ds_trg.nav_lon.values.squeeze()
lat_trg=ds_trg.nav_lat.values.squeeze()
lon_trg_rad=np.radians(lon_trg)
lat_trg_rad=np.radians(lat_trg)

print('ni/nj out')
nj_out=lon_trg.shape[0]
ni_out=lon_trg.shape[1]

mask_trg=np.zeros(shape=(nj_out,ni_out+2))
mask_coast_trg=np.zeros(shape=(nj_out,ni_out))

print('get coastal mask')
ssmask_trg=ds_trg.tmaskutil.values[0,:,:].squeeze()
mask_trg[:,1:-1]=ds_trg.tmask.values[0,0,:,:].squeeze()
mask_trg[:, 0]=mask_trg[:,-2]
mask_trg[:,-1]=mask_trg[:, 1]

mask_coast_trg[1:-1,:]= ( mask_trg[1:-1,1:-1] + 
                          mask_trg[0:-2,1:-1] + mask_trg[2::,1:-1] + mask_trg[1:-1,0:-2] + mask_trg[1:-1,2::] + 
                          mask_trg[0:-2,0:-2] + mask_trg[2::,2::]  + mask_trg[2::,0:-2]  + mask_trg[0:-2,2::] ) * mask_trg[1:-1,1:-1]
mask_coast_trg[(mask_coast_trg > 1) & (mask_coast_trg < 9)] = 10
mask_coast_trg[mask_coast_trg != 10] = np.nan
mask_coast_trg=mask_coast_trg*ssmask_trg.astype(np.float32)
mask_coast_trg[mask_coast_trg==10]=1

print('get 1d array of coastal points')
iy_trg,ix_trg=np.where(mask_coast_trg==1)
mask_coast_trg_1d=mask_coast_trg[iy_trg,ix_trg]
lon_trg_rad_1d=lon_trg_rad[iy_trg,ix_trg]
lat_trg_rad_1d=lat_trg_rad[iy_trg,ix_trg]

open output file
ni/nj out
get coastal mask
get 1d array of coastal points


In [9]:
hicb_trg=np.zeros(shape=(nj_out,ni_out))
ij_miss=np.where(mask_coast_trg == 1)
print(len(ij_miss[0]))
for ipt,pts in enumerate(ij_miss[0]):
    if ipt%100 == 0:
        print(ipt)
    iiy=ij_miss[0][ipt]
    iix=ij_miss[1][ipt]
    lon_trg_rad_0d=lon_trg_rad[iiy,iix]
    lat_trg_rad_0d=lat_trg_rad[iiy,iix]
    dist=get_dist_ocean_point_1d(lon_trg_rad_0d, lat_trg_rad_0d, lon_in_1d, lat_in_1d)
    if np.min(dist) < 100 :
        hicb_trg[iiy,iix]=np.mean(h_in_1d[dist<=np.min(dist)+5])
        if hicb_trg[iiy,iix]== 0:
            print(iiy,iix,h_in_1d[dist<=np.min(dist)+5])
    

3912
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900


In [10]:
print('write output')
# ## Write output
# define data with variable attributes
data_vars = {'icbh':(['y','x'], hicb_trg,
                          {'units': 'm', 
                           'long_name':'iceberg maximal thickness'}),
             'coast':(['y','x'], mask_coast_trg,
                         {'units': '[]', 
                           'long_name':'coastl point mask'})
                          }

# define coordinates
coords = {'nav_lon': (['y','x'], lon_trg),
          'nav_lat': (['y','x'], lat_trg)}

# define global attributes
attrs = {'source':cfin, 
         'method':('Average of all BedMachine coastal points in a circle of d km center on each coastal NEMO T points.'
         'd = 5 + min(gread_circle_distance(NEMO T point, List of BedMachine coastal points))')
         }

# create dataset
ds_out = xr.Dataset(data_vars=data_vars, 
                coords=coords, 
                attrs=attrs)

ds_out.to_netcdf(cfout)

write output


In [11]:
ds_calv=xr.open_dataset('calving_eANT025.L121.nc')

In [12]:
ds_calv

<xarray.Dataset>
Dimensions:   (time_counter: 1, y: 439, x: 1440)
Dimensions without coordinates: time_counter, y, x
Data variables:
    soicbclv  (time_counter, y, x) float32 ...
Attributes:
    start_date:        -1
    output_frequency:  N/A
    CONFIG:            N/A
    CASE:              N/A
    history:           Fri Aug 19 15:21:19 2022: ncks -d y,0,438 -d x,1,1440 ...
    NCO:               netCDF Operators version 5.0.6 (Homepage = http://nco....

In [13]:
ij_clv=np.where(ds_calv.soicbclv[0,:,:]!= 0)
for ipt, pts in enumerate(ij_clv[0]):
    iiy=ij_clv[0][ipt]
    iix=ij_clv[1][ipt]
    if hicb_trg[iiy,iix]==0 :
        print(iiy,iix,ds_calv.soicbclv.values[:,iiy,iix],hicb_trg[iiy,iix],mask_coast_trg[iiy,iix])

In [14]:
ds_bm=xr.open_dataset('BedMachineAntarctica-v3.nc')

In [15]:
hdbg=ds_bm.thickness.values[ds_bm.mask.values > 1]

In [16]:
np.min(hdbg)

0.12693787